# Time travel
- Doc: https://langchain-ai.github.io/langgraph/tutorials/get-started/6-time-travel/

In [ ]:
from dotenv import load_dotenv

# API-KEY 읽어오기
load_dotenv()

In [ ]:
from langchain.chat_models import init_chat_model

# 모델 초기화
llm = init_chat_model("google_genai:gemini-2.5-flash")

## 1. Rewind your graph

In [ ]:
from typing import Annotated

from langchain_tavily import TavilySearch
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


# 에이전트의 상태 정의 클래스
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 상태 기반 워크플로우 생성
graph_builder = StateGraph(State)

# 웹 검색을 도구
tool = TavilySearch(max_results=2)
# 도구 리스트
tools = [tool]

# LLM이 도구 호출 여부 판단
llm_with_tools = llm.bind_tools(tools)

# chatbot 노드 함수
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 워크플로우에 chatbot 노드 추가
graph_builder.add_node("chatbot", chatbot)

# tool 노드 워크플로우에 추가
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

# 조건부 라우팅: tools로 이동하거나 END로 이동 (종료)
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools 노드 실행 후 chatbot 노드로 다시 이동 (도구 결과 처리)
graph_builder.add_edge("tools", "chatbot")

# 워크플로우 시작점에서 chatbot 노드로 이동
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
# 워크플로우 컴파일, 실행 가능한 워크플로우 생성.
# memory checkpointer 사용
graph = graph_builder.compile(checkpointer=memory)

## 2. Add steps

In [ ]:
# 대화 세션을 구분하기 위한 설정
config = {"configurable": {"thread_id": "1"}}

# 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "LangGraph를 배우고 있어."
                    "LangGraph에 대해 좀 조사해 줄래?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
# 워크플로우를 스트리밍 모드로 실행 (사용자 입력 처리)
# 이전 답변에 이어서 대화 추가
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "오, 도움이 됐어. "
                    "이걸로 자율 에이전트를 만들어 봐야지!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

## 3. Replay the full state history

In [ ]:
# config에 해당하는 워크플로우의 상태 스냅샷 조회
snapshot = graph.get_state(config)
# 스냅샷 메시지를 순서대로 출력
for m in snapshot.values["messages"]:
    m.pretty_print()

In [ ]:
# 재생할 상태를 저장할 변수
to_replay = None

# config에 해당하는 워크플로우의 상태 히스토리
history = graph.get_state_history(config)
for state in history:
    # state.next는 다음에 실행할 노드
    print("메지시 수: ", len(state.values["messages"]), "다음 실행 노드: ", state.next)
    print("-" * 80)

    # 메시지 수가 2인 상태를 찾아 to_replay에 저장 (임의로 선택)
    if len(state.values["messages"]) == 2:
        to_replay = state

In [ ]:
# to_replay의 다음 노드 출력
print(to_replay.next)
# to_replay의 설정 객체 출력
print(to_replay.config)
to_replay

## 4. Load a state from a moment-in-time

In [ ]:
# to_replay.config의 checkpoint_id를 사용하여 워크플로우 상태 재생 (스트리밍 모드)
events = graph.stream(None, to_replay.config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

In [ ]:
# config에 해당하는 워크플로우의 상태 스냅샷 조회 (변경된 내용 확인)
snapshot = graph.get_state(config)
# 스냅샷 메시지를 순서대로 출력
for m in snapshot.values["messages"]:
    m.pretty_print()